In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# setup disply parameters
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from matplotlib.ticker import StrMethodFormatter
float_formatter = StrMethodFormatter('{x:0.03f}')
from IPython.core.display import display, HTML
from pprint import pprint
display(HTML("<style>.container { width:95% !important; }</style>"))
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=(18, 6)) # set figure size

## Run a session of the SCML world (2020)

The SCML world (Supply Chain Management League) runs on top of `NegMAS`.

In this tutorial, you will test a run of this world.

Firstly, let's import everything from the SCML app

In [2]:
from scml.scml2020 import *

There are several ways to create an SCML world. One of the simplest is to use one of the class methods of the `SCML2020World` class. 


In [14]:
world = SCML2020World(**SCML2020World.generate(agent_types=[DecentralizingAgent], n_steps=50))

*If you want to just test your installation (and do not care whether you get an accurate indicator of agent performance), you can set the number of steps to a small value (e.g. n_steps=10).*

Now you can run this world simulation by just calling `run`.

In [ ]:
world.run()

Let's see what happened in this run. Firstly, how many negotiations were conducted over time

In [ ]:
plt.plot(world.stats['n_negotiations'])
plt.xlabel('Simulation Step')
plt.ylabel('N. Negotiations')
plt.show()

It is clear that many negotiations happened at the beginning of the simulation with smaller number later. That is expected as the consumers publish their calls for proposals early on.

Several other market statistics are available:

In [ ]:
pprint(list(_ for _ in world.stats.keys() if "@" not in _ and not _.startswith("_") ))

We can for example check the activity level of this world (defined as the total amount of money transferred due to trade)

In [ ]:
plt.plot(world.stats['activity_level'])
plt.xlabel('Simulation Step')
plt.ylabel('Activitiy Level ($)\nTotal Money Transferred')
plt.show()

We can see a picture of contracting in this world as follows:

In [ ]:
plt.plot(world.stats['n_contracts_concluded'], label='Concluded Contracts')
plt.plot(world.stats['n_contracts_cancelled'], label='Cancelled Contracts') 
plt.plot(world.stats['n_contracts_signed'], label='Signed Contracts') 
plt.plot(world.stats['n_contracts_executed'], label='Executed Contracts')
plt.legend()
plt.xlabel('Simulation Step')
plt.ylabel('N. Contracts')
plt.show()

We can also check the breaches that happened

In [ ]:
plt.plot(world.stats['breach_level'])
plt.xlabel('Simulation Step')
plt.ylabel('Total Breach Level')
plt.show()

It is also possible to focus on the behavior of some specific agent. Let's first find who was the winner (the factory manager that got maximum wealth). Notice that there can be multiple winners

In [ ]:
winner_profits = [round(100.0 * world.stats[f'balance_{_}'][-1] / world.stats[f'balance_{_}'][0] - 100.0, 2) for _ in world.winners]
print(f"{world.winners} won at {winner_profits}%")

What was the balance of this winner over time:

In [ ]:
for winner in world.winners:
    plt.plot(world.stats[f'balance_{winner}'], label=f'{winner}\n({winner.__class__.__name__})')
plt.legend()
plt.xlabel('Simulation Step')
plt.ylabel('Winner\'s Balance ($)')
plt.show()

Let's check how did its total storage change over time:

In [ ]:
for winner in world.winners:
    plt.plot(world.stats[f'storage_{winner}'], label=winner)
plt.xlabel('Simulation Step')
plt.ylabel('Player\'s Total Storage (item)')
plt.legend()
plt.show()

We can actually check what happens to ALL competitors:

In [ ]:
for name, _ in sorted(((_.name, _) for _ in world.agents.values() if _.name != "SYSTEM"), key=lambda x: x[0]):    
    plt.plot(np.asarray(world.stats[f'balance_{name}'])/world.stats[f'balance_{name}'][0], label=name)
plt.xlabel('Simulation Step')
plt.ylabel('Player Balance ($)')
plt.legend()
plt.show()